In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL da página
url = "https://fbref.com/pt/comps/24/cronograma/Serie-A-Resultados-e-Calendarios"
# Faz a requisição para a página
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Função para extrair a tabela da página
def extrair_tabela(rows):
    headers = []
    data = []
    
    for row in rows:
        cols = row.find_all(['th', 'td'])
        if len(headers) == 0:  # A primeira linha será o cabeçalho
            headers = [col.get_text
                       (strip=True) for col in cols]
        else:  # Demais linhas são dados
            data.append([col.get_text(strip=True) for col in cols])
    
    # Cria o DataFrame
    return pd.DataFrame(data, columns=headers)

# Encontra o container da tabela
table_container = soup.find('div', {'class': 'table_container'})
rows = table_container.find_all('tr')

# Extrai a tabela
tabela = extrair_tabela(rows)

# Encontra a coluna "Relatório da Partida"
links_relatorios = []
for row in rows:
    a_tag = row.find('a', text='Relatório da Partida')
    if a_tag:
        href = a_tag.get('href')
        if href and href != "#":  # Ignora links vazios ou "#" como marcador
            links_relatorios.append(href)
        else:
            links_relatorios.append(None)
    else:
        links_relatorios.append(None)

# Remove linhas onde a coluna "Relatório da Partida" é "Confronto"
tabela = tabela[~tabela['Relatório da Partida'].str.contains("Confronto", na=False)]

# Adiciona a coluna de links na tabela
tabela['Link Relatório'] = links_relatorios[:len(tabela)]  # Ajusta o tamanho da lista para o tamanho da tabela

C:\Users\eduar\AppData\Local\Temp\ipykernel_15084\3731441437.py:37: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  a_tag = row.find('a', text='Relatório da Partida')


In [2]:
# Desloca a coluna "Link Relatório" uma linha para cima
tabela['Link Relatório'] = tabela['Link Relatório'].shift(-1)

# Preenche a última linha da coluna "Link Relatório" com NaN (caso não esteja automaticamente)
tabela.loc[tabela.index[-1], 'Link Relatório'] = None


In [3]:
# Remove linhas onde "Link Relatório" é None
tabela = tabela[tabela['Link Relatório'].notna()]

In [4]:
tabela

,Sem,Dia,Data,Horário,Em casa,xG,Resultado,xG,Visitante,Público,Local,Árbitro,Relatório da Partida,Notas,Link Relatório
0,1,sáb,2024-04-13,18:30,Criciúma,0.7,1–1,0.6,Juventude,12.804,Estádio Heriberto Hülse,,Relatório da Partida,,/pt/partidas/60ef737d/Criciuma-Juventude-2024A...
1,1,sáb,2024-04-13,18:30,Internacional,1.9,2–1,0.7,Bahia,19.103,Estádio José Pinheiro Borba,,Relatório da Partida,,/pt/partidas/620b512a/Internacional-Bahia-2024...
2,1,dom,2024-04-14,16:00,Ath Paranaense,2.0,4–0,0.3,Cuiabá,21.388,Ligga Arena,,Relatório da Partida,,/pt/partidas/086cf01e/Athletico-Paranaense-Cui...
3,1,dom,2024-04-14,16:00,Vasco da Gama,0.9,2–1,1.2,Grêmio,17.722,Estádio Club de Regatas Vasco da Gama,,Relatório da Partida,,/pt/partidas/865e8350/Vasco-da-Gama-Gremio-202...
4,1,dom,2024-04-14,16:00,Corinthians,0.5,0–0,0.3,Atlético Mineiro,44.285,Neo Química Arena,,Relatório da Partida,,/pt/partidas/d3926429/Corinthians-Atletico-Min...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,28,dom,2024-09-29,18:30,Cruzeiro,0.7,1–1,1.2,Vasco da Gama,,Estádio Governador Magalhães Pinto,Matheus Delgado Candançan,Relatório da Partida,,/pt/partidas/26feb496/Cruzeiro-Vasco-da-Gama-2...
314,28,dom,2024-09-29,18:30,Internacional,2.3,3–1,0.7,Vitória,,Estádio José Pinheiro Borba,Wagner do Nascimento Magalhães,Relatório da Partida,,/pt/partidas/7b02ac23/Internacional-Vitoria-20...
315,28,dom,2024-09-29,18:30,Bahia,2.4,1–0,0.1,Criciúma,,Itaipava Arena Fonte Nova,Edina Alves,Relatório da Partida,,/pt/partidas/b2e0b425/Bahia-Criciuma-2024Setem...
316,28,dom,2024-09-29,20:00,Flamengo,0.7,1–0,0.4,Ath Paranaense,,Estadio Jornalista Mário Filho,Davi de Oliveira Lacerda,Relatório da Partida,,/pt/partidas/b3539655/Flamengo-Athletico-Paran...


In [5]:
tabela.to_csv('C://Users//eduar//Desktop//DEV//Football//Brasileirao//links.csv', index= False)